In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
1,170,SI,VIGENTE,TRÁNSITO RESTRINGIDO,27/01/2023,02/02/2023 21:00 HORAS,CUSCO/QUISPICANCHI/ANDAHUAYLILLAS,KM 1010,LA VIA AFECTADA POR UN GRUPO DE PERSONAS QUE ...,-13.6743076,...,-13.6743076+-71.66972082,2023 21:00 HORAS+02+02+,0,0,CEOPOL,HUMANO,PE-3S,LONGITUDINAL DE LA SIERRA SUR,SI,restringido
2,29,SI,VIGENTE,TRÁNSITO RESTRINGIDO,23/01/2023,02/02/2023 21:00 HORAS,APURIMAC/ABANCAY/PICHIRHUA,417,LA VIA AFECTADA POR UN GRUPO DE PERSONAS QUE ...,-13.884221,...,-13.884221+-73.031517,2023 21:00 HORAS+02+02+,0,0,CONCESIONARIA SURVIAL,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
3,49,SI,VIGENTE,TRÁNSITO RESTRINGIDO,17/01/2023,02/02/2023 21:00 HORAS,APURIMAC/AYMARAES/COTARUSE,KM 309,LA VIA AFECTADA POR UN GRUPO DE PERSONAS QUE ...,-14.500474,...,-14.500474+-73.25867,2023 21:00 HORAS+02+02+,0,0,DESPRCAR PNP CHALHUANCA,HUMANO,PE-30A,CARRETERA NASCA - PUQUIO - ABANCAY,SI,restringido
6,230,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,31/01/2023,02/02/2023 21:00 HORAS,JUNIN/CHANCHAMAYO/PICHANAQUI,KM 69+200,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-10.92402,...,-10.92402+-74.877533,2023 21:00 HORAS+02+02+,3,1,COE-MTC,HUMANO,PE-5S,LONGITUDINAL DE LA SELVA SUR,SI,interrumpido
7,236,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,28/01/2023,02/02/2023 21:00 HORAS,JUNIN/YAULI/MOROCOCHA,KM 138-140,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-11.596887,...,-11.596887+-76.1443872,2023 21:00 HORAS+02+02+,4,77,DEVIANDES,HUMANO,PE-22,CARRETERA CENTRAL,SI,interrumpido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,307,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,02/02/2023 21:00 HORAS,PUNO/CHUCUITO/ZEPITA,KM 287,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.6042637,...,-16.6042637+-69.1857072,2023 21:00 HORAS+02+02+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
93,316,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,02/02/2023 21:00 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 310,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.72557,...,-16.72557+-69.67592,2023 21:00 HORAS+02+02+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
94,312,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,04/01/2023,02/02/2023 21:00 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 0,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.730059545472,...,-16.730059545472+-69.71403699189,2023 21:00 HORAS+02+02+,0,0,CEOPOL,HUMANO,PE-38A,VIA MAZOCRUZ - ILAVE,SI,interrumpido
95,313,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,05/01/2023,02/02/2023 21:00 HORAS,PUNO/EL COLLAO/SANTA ROSA,KM 175 AL 213,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.7501666,...,-16.7501666+-69.7209405,2023 21:00 HORAS+02+02+,0,0,CEOPOL,HUMANO,PE-36A,CARRETERA MOQUEGUA - TORATA - MAZOCRUZ - DESAG...,SI,interrumpido
